# 1.) Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('/content/drive/MyDrive/BrainAgeRegression')

In [ ]:
from utils.utils import BrainAgeDataset, set_seed, count_parameters, split_dataset, normalize_targets, denormalize_targets

In [ ]:
from utils.train_utils import BrainAgeTrainer
from utils.eval_utils import BrainAgeEvaluator

In [ ]:
csv_path = '/content/drive/MyDrive/BrainAgeRegression/matched_metadata.csv'
df = pd.read_csv(csv_path)

nifti_dir = '/content/drive/MyDrive/BrainAgeRegression/data/nifti'
full_dataset = BrainAgeDataset(df, nifti_dir)


In [ ]:
sys.path.append('/content/drive/MyDrive/BrainAgeRegression/models')

In [ ]:
from cnn3d import Simple3DCNN

# 2.) Normalize Data

In [ ]:
df, mean_age, std_age = normalize_targets(df, target_col='Age')
full_dataset = BrainAgeDataset(df, nifti_dir, use_normalized_age=True)

# 2.) Train/Test/Val Split

In [ ]:
train_dataset, val_dataset, test_dataset = split_dataset(full_dataset)

In [ ]:
print(f"📊 Train size: {len(train_dataset)}")
print(f"📊 Val size:   {len(val_dataset)}")
print(f"📊 Test size:  {len(test_dataset)}")

# 3.) Initialize our Model and Components

In [ ]:
model = Simple3DCNN()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(model)

# 4.) Create our DataLoaders

In [ ]:
batch_size = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

# 5.) Training Loop

In [ ]:
trainer = BrainAgeTrainer(model, train_loader, val_loader, criterion, optimizer, device)
trainer.train(epochs=10, track_predictions=True)

history = trainer.get_history()
train_pred, train_true = trainer.get_predictions()['train']
val_pred, val_true = trainer.get_predictions()['val']

# 6.) Test Eval

In [ ]:
evaluator = BrainAgeEvaluator(model, device)
metrics, y_pred, y_true = evaluator.evaluate(test_loader, criterion)

print(f"📏 MAE: {metrics['mae']:.2f} | R²: {metrics['r2']:.3f} | RMSE: {metrics['rmse']:.2f}")

# 7.) Evaluate our Model

In [ ]:
evaluator.plot_predictions(y_true, y_pred, title="🧪 Test Set Predictions", color='green')
evaluator.plot_predictions(train_true, train_pred, title="🏋️ Train Set Predictions", color='blue')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Train plot
axes[0].scatter(train_true, train_pred, alpha=0.5, color='blue')
axes[0].plot([min(train_true), max(train_true)], [min(train_true), max(train_true)], 'r--')
axes[0].set_title("🏋️ Train Set")
axes[0].set_xlabel("True Age")
axes[0].set_ylabel("Predicted Age")
axes[0].grid(True)

# Test plot
axes[1].scatter(y_true, y_pred, alpha=0.8, color='green')
axes[1].plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--')
axes[1].set_title("🧪 Test Set")
axes[1].set_xlabel("True Age")
axes[1].grid(True)

plt.suptitle("🧠 Predicted vs. True Age (Train vs. Test)", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
